# collect data from ppt beauty

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
import random
import re
from concurrent.futures import ThreadPoolExecutor
import threading
import sys
payload = {
    'from':'/bbs/Beauty/index.html',
    'yes':'yes'
}
rs = requests.session()
res = rs.post("https://www.ptt.cc/ask/over18",data = payload)

# Method 1 : 以推文數量作為照片的分數
* get image links from ptt

In [2]:

# get image links from ptt
image_score = {i : [] for i in range(0,101)}
lock = threading.Lock()
def get_image(index):
    global image_score, lock
    content = rs.get(f"https://www.ptt.cc/bbs/Beauty/index{index}.html")
    soup = BeautifulSoup(content.text, 'html.parser')
    articles = soup.select('div.r-ent')
    for article in articles:
        article_tag = article.select_one(".nrec").text
        if article_tag=="":
            continue
        url = "https://www.ptt.cc/" +article.select_one('.title').a['href']
        content = rs.get(url)
        soup = BeautifulSoup(content.text, 'html.parser')
        main = soup.select_one("div#main-content")
        excluded_tag = main.select(".push,.f2,div.article-mataline,.article-mataline-right")
        for tag in excluded_tag:
            tag.decompose()
        all_link = main.select("a")
        for link in all_link:
            if re.match(r'.*\.(jpg|png|jpeg)$',link['href']):
                with lock:
                    if article_tag=="爆":
                        image_score[100].append(link['href'])
                    elif not article_tag.isnumeric():
                        image_score[0].append(link['href'])
                    else:
                        image_score[int(article_tag)].append(link['href'])
        delay_choices = [0.2,0.3,0.4,0.5] 
        delay = random.choice(delay_choices) 
        time.sleep(delay)
with ThreadPoolExecutor(max_workers=10) as executor:
    for i in range(3450,4002):
        executor.submit(get_image,i)
output1 = "image_score.json"
# output2 = "not_popular_image.json"
with open(output1,'w') as f:
    json.dump(image_score,f)
# with open(output2,'w') as f:
#     json.dump({
#         "image_urls":not_popular_image
#     },f)

### 不同分數的照片數量

In [3]:
image_len = {i:len(image_score[i]) for i in range(0,101)}
image_len

{0: 1299,
 1: 7303,
 2: 5633,
 3: 4433,
 4: 3856,
 5: 3279,
 6: 3216,
 7: 3211,
 8: 2594,
 9: 2784,
 10: 1673,
 11: 1149,
 12: 1299,
 13: 1625,
 14: 1525,
 15: 1531,
 16: 1253,
 17: 1549,
 18: 1378,
 19: 1195,
 20: 1122,
 21: 1371,
 22: 1099,
 23: 990,
 24: 971,
 25: 820,
 26: 955,
 27: 1023,
 28: 709,
 29: 849,
 30: 766,
 31: 687,
 32: 633,
 33: 468,
 34: 666,
 35: 608,
 36: 484,
 37: 516,
 38: 519,
 39: 499,
 40: 530,
 41: 530,
 42: 635,
 43: 443,
 44: 480,
 45: 392,
 46: 462,
 47: 689,
 48: 415,
 49: 339,
 50: 381,
 51: 306,
 52: 220,
 53: 368,
 54: 399,
 55: 145,
 56: 256,
 57: 201,
 58: 206,
 59: 192,
 60: 172,
 61: 194,
 62: 197,
 63: 241,
 64: 143,
 65: 249,
 66: 246,
 67: 187,
 68: 166,
 69: 200,
 70: 132,
 71: 261,
 72: 110,
 73: 145,
 74: 129,
 75: 179,
 76: 125,
 77: 174,
 78: 100,
 79: 58,
 80: 81,
 81: 155,
 82: 130,
 83: 141,
 84: 103,
 85: 183,
 86: 72,
 87: 172,
 88: 114,
 89: 75,
 90: 162,
 91: 152,
 92: 181,
 93: 25,
 94: 81,
 95: 119,
 96: 64,
 97: 101,
 98: 23,
 99:

### 讀取圖片連結，轉為圖檔
* 確定圖檔可以被開啟

In [5]:
import json
import urllib.request
from pathlib import Path
from PIL import Image
import os
with open("./image_score.json") as f:
    image_score = json.load(f)
#get image from links
def get_image(i):
    data = image_score[str(i)]
    folder_path = f"data/{i}/"
    Path(folder_path).mkdir(parents=True, exist_ok=True)
    for i in range(len(data)):
        try:
            url = data[i]
            filename = Path(folder_path)/data[i].split("/")[-1]
            # print(filename)
            # break
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
# check if image can be read by Image.open()  
def detect_image(i):
    folder = f"data/{i}"
    image_list = Path(folder).glob("*")
    for image in image_list:
        try:
            img = Image.open(str(image))
        except Exception as err:
                print("err:",err)
                print("image:",image)
                os.remove(image)
with ThreadPoolExecutor(max_workers = 10) as executor:
    for i in range(21,101):
        executor.submit(get_image,i)
with ThreadPoolExecutor(max_workers = 10) as executor:
    for i in range(0,101):
        executor.submit(detect_image,i)


Sth. wrong with https://upload.cc/i1/2021/06/27/jaGxI7.png, error : HTTP Error 403: Forbidden
Sth. wrong with https://upload.cc/i1/2021/06/27/IY2GHj.png, error : HTTP Error 403: Forbidden
Sth. wrong with https://pbs.twimg.com/media/E_njXVYUcAAy6nC.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://pbs.twimg.com/media/E_VONIPVEAMGnVs.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://pbs.twimg.com/media/E_paAAjVgAEBDBQ.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://pbs.twimg.com/media/E-h93dvVQAsed32.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://pbs.twimg.com/media/E6379VDUYAkP-Wz.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://pbs.twimg.com/media/E6379VUVUAglLr6.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://upload.cc/i1/2022/04/25/lwHVGy.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://assets.juksy.com/files/articles/110358/800x_100_w-60ff6020034c7.jpg, error : HTTP Error 403: Forbidd

### 建立train與test data folder

In [10]:
import os
import shutil 
def create_train_test(index):
    src_folder = f"./data/{index}"
    train_folder = f"new_data/train/{index}"
    test_folder = f"new_data/test/{index}"
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

# loop through all files in the source folder
    img_list =os.listdir(src_folder)
    for i,filename in enumerate(img_list):
        if i < (len(img_list)/5):
            src_path = os.path.join(src_folder, filename)
            dst_path = os.path.join(test_folder, filename)
            shutil.move(src_path, dst_path)
        else:
            src_path = os.path.join(src_folder, filename)
            dst_path = os.path.join(train_folder, filename)
            shutil.move(src_path, dst_path)
with ThreadPoolExecutor(max_workers = 10) as executor:
    for i in range(0,101):
        executor.submit(create_train_test,i)


In [12]:
for i in range(0,101):
    for mode in ['train','test']:
    
        l = len(list(Path(f"new_data/{mode}/{i}").glob("*")))
        print(mode,i,l)

train 0 1022
test 0 256
train 1 5804
test 1 1452
train 2 4432
test 2 1108
train 3 3500
test 3 876
train 4 3049
test 4 763
train 5 2590
test 5 648
train 6 2533
test 6 634
train 7 2424
test 7 606
train 8 2064
test 8 517
train 9 2212
test 9 554
train 10 1336
test 10 334
train 11 908
test 11 228
train 12 1018
test 12 255
train 13 1296
test 13 324
train 14 1215
test 14 304
train 15 1224
test 15 307
train 16 998
test 16 250
train 17 1237
test 17 310
train 18 1099
test 18 275
train 19 956
test 19 239
train 20 895
test 20 224
train 21 1096
test 21 274
train 22 877
test 22 220
train 23 792
test 23 198
train 24 776
test 24 195
train 25 655
test 25 164
train 26 759
test 26 190
train 27 799
test 27 200
train 28 567
test 28 142
train 29 664
test 29 167
train 30 612
test 30 154
train 31 548
test 31 138
train 32 506
test 32 127
train 33 370
test 33 93
train 34 518
test 34 130
train 35 486
test 35 122
train 36 383
test 36 96
train 37 412
test 37 104
train 38 414
test 38 104
train 39 399
test 39 100
tr

### 從圖片上將臉截取下來
* 方法一　: 用face_recognition 套件
問題:face_recognition package 安裝失敗
* 方法二 :　用python-opencv

In [14]:
import face_recognition
for i in range(0,101):
    train_image_list = list(Path(f"new_data/train/{i}/").glob("*"))
    test_image_list = list(Path(f"new_data/test/{i}/").glob("*"))
    for l in [train_image_list,test_image_list]:
        for image in l:
            img = face_recognition.load_image_file(image)
            face_locations = face_recognition.face_locations(img)
            if len(face_locations) != 1 : #若沒有或超過一張臉，就捨棄該圖檔
                # print("not a good image")
                os.remove(image)
                continue
            else:
                top, right, bottom, left = face_locations[0]
                face_img = img[top:bottom, left:right]
                pil_image = Image.fromarray(face_img)
                pil_image.save(image)

ModuleNotFoundError: No module named 'face_recognition'

In [10]:
import cv2
for i in range(0,101):
    train_image_list = list(Path(f"new_data/train/{i}/").glob("*"))
    test_image_list = list(Path(f"new_data/test/{i}/").glob("*"))
    for l in [train_image_list,test_image_list]:
        for image in l:
            img = cv2.imread(str(image))
            # crop face from image
            face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_detector.detectMultiScale(img)
            if len(faces)>=1:
                x, y, w, h = faces[:1][0]
                pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                face_bbox = (x, y, x+w, y+h)
                target_img = pil_img.crop(face_bbox)

            else:
                target_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            target_img.save(image)

# Method 2 : 將圖片分為popular與not popular兩種
* 將link轉為圖檔
* 同時建立train、test data folder

In [32]:
import json
import urllib.request
from pathlib import Path
from urllib.error import HTTPError
with open("./popular_image.json") as f:
    popular_image = json.load(f)
with open("./not_popular_image.json") as f:
    not_popular_image = json.load(f)
not_popular_image = not_popular_image['image_urls'][:15000]
popular_image = popular_image['image_urls'][:15000]

for i in range(len(popular_image)):
    if i<(len(popular_image)*4/5):
        try:
            url = popular_image[i]
            filename = "train/popular/"+popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
        # continue
    else:
        try:
            url = popular_image[i]
            filename = "test/popular/"+popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
for i in range(len(not_popular_image)):
    if i<(len(not_popular_image)*4/5):
        try:
            url = not_popular_image[i]
            filename = "train/not_popular/" + not_popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
            
    else:
        try:
            url = not_popular_image[i]
            filename = "test/not_popular/"+not_popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")

Sth. wrong with https://imgur.dcard.tw/IIwP9Y4h.png, error : HTTP Error 403: Forbidden
Sth. wrong with https://imgur.com/undefined.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://i2.kknews.cc/SIG=1mjs6kc/1ns40000q02nopp1n5s5.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://i1.kknews.cc/SIG=318r8na/19sr00069o3rq6so2706.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://img.bingfeng.tw/bingfeng/forum/201811/19/210508q7x70trrmvsmotjr.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://assets.juksy.com/files/articles/91916/800x_100_w-5d11dea5c11ba.png, error : HTTP Error 403: Forbidden
Sth. wrong with https://file.jichengguandao.com/uploads/2020/07/07/nzuxf25ot0m.png, error : <urlopen error [WinError 10054] 遠端主機已強制關閉一個現存的連線。>
Sth. wrong with https://img.chinatimes.com/newsphoto/2019-05-08/656/20190508001595.jpg, error : <urlopen error [Errno 11001] getaddrinfo failed>
Sth. wrong with https://img.chinatimes.com/newsphoto/2019-05-08/656/201

In [35]:
import os
import glob
mode = ["train", "test"]
t= ["popular", "not_popular"]
for m in mode:
    for i in t:
        folder = f"./{m}/{i}/"
        data = glob.glob(os.path.join(folder,'*'))
        print(f"{m} {i}:",len(data))

train popular: 11807
train not_popular: 11522
test popular: 2985
test not_popular: 2980


In [ ]:
with open("./not_popular_image.json") as f:
    not_popular_image = json.load(f)
not_popular_image = not_popular_image['image_urls'][:15000]
for image in not_popular_image:
    try:
        img = Image.open(image)
    except Exception as err:
        print(image)